In [3]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Table
from sqlalchemy import create_engine, inspect
from sqlalchemy import MetaData
from edgerdb import helper_functions as hlp
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import re
from requests import get

In [2]:
meta = MetaData()

engine = create_engine('postgresql://analyst:@localhost:5432/edgar')

messages = Table('filings', meta, autoload=True, autoload_with=engine)

def sql_to_df(query, connection):
    df = pd.read_sql(query, connection)
    connection.close()
    return df


In [ ]:
def create_csi_data_table(engine):
    '''
    Creates a table with the data from csidata.com
    '''
    conn = engine.connect()
    csi_stock_data = pd.read_csv('http://www.csidata.com/factsheets.php?type=stock&format=csv')
    csi_stock_data.columns = map(str.lower, csi_stock_data.columns)
    csi_stock_data = csi_stock_data.rename(columns={'name': 'company_name'})
    csi_stock_data['company_name'] = csi_stock_data['company_name'].str.upper()
    csi_stock_data['company_name'] = csi_stock_data['company_name'].str.strip()
    csi_stock_data = csi_stock_data.fillna(value="NONE")
    csi_stock_data.to_sql('csi_stock_data', engine)
    conn.close()
    
def create_table_with_cik_and_csi_data():
    '''
        Creates a table called cik_to_csi by joining filings and csi_stock_data tables on company_name
    '''
    join_ticker_to_cik = """select * into temp_cik_to_csi from (select fil.cik, fil.company_name,
                            csi.symbol, csi.exchange,
                            csi.isactive, csi.startdate, csi.enddate
                            from filings fil
                            join csi_stock_data csi on fil.company_name = csi.company_name) as foo;"""
    clean_ticker_to_cik_table = """select * into cik_to_csi
    from (select distinct cik, c.company_name, c.symbol, c.exchange, c.isactive, c.startdate, c.enddate  from temp_cik_to_csi c) as foo;"""
    hlp.clear_sessions('edgar', hlp.db())
    hlp.statement(join_ticker_to_cik, hlp.db(), output=False, commit=True)
    hlp.statement(clean_ticker_to_cik_table, hlp.db(), output=False, commit=True)
    hlp.statement("drop table temp_cik_to_csi;", hlp.db(), output=False, commit=True)
    


In [1]:
def tickers_to_cik(list_of_tickers):
    URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
    CIK_RE = re.compile('.*CIK=(\d{10}).*')

    cik_dict = {}
    for ticker in list_of_tickers:
        results = CIK_RE.findall(get(URL.format(ticker)).text)
        if len(results):
            cik_dict[str(ticker).upper()] = str(results[0]).lstrip('0')
        else:
            no_matches.append(str(ticker).upper())
    return cik_dict

def new_nasdaq_tickers_and_names(engine):
    """
        This function queries the cik_to_csi table and returns a dataframe of tickers and company names for companies
        not found in our data.
    """
    nasdaq_companies = pd.read_csv("data/nasdaqcompanylist.csv")
    amex_companies = pd.read_csv("data/amexcompanylist.csv")
    nyse_companies = pd.read_csv("data/nysecompanylist.csv")
    nasdaq_name_and_ticker = pd.DataFrame(nasdaq_companies, columns=['Symbol', 'Name'])
    nasdaq_name_and_ticker['exchange'] = 'NASDAQ'
    amex_name_and_ticker = pd.DataFrame(amex_companies, columns=['Symbol', 'Name'])
    amex_name_and_ticker['exchange'] = 'AMEX'
    nyse_name_and_ticker = pd.DataFrame(nyse_companies, columns=['Symbol', 'Name'])
    nyse_name_and_ticker['exchange'] ='NYSE'
    companies_and_symbols = pd.concat([nasdaq_name_and_ticker, amex_name_and_ticker, nyse_name_and_ticker])
    companies_and_tickers = companies_and_symbols.rename(columns={'Symbol': 'symbol', "Name": "name"})
    companies_and_tickers['isactive'] = 1
    list_of_tickers_from_nasdaq = companies_and_tickers['symbol'].values.tolist()
    list_of_tickers_in_cik_to_csi_table = pd.read_sql_query('select distinct symbol from cik_to_csi;',con=engine)['symbol'].values.tolist()
    not_found_in_db = []
    for x in list_of_tickers_from_nasdaq:
        if x not in list_of_tickers_in_cik_to_csi_table and x not in not_found_in_db:
            not_found_in_db.append(x)
    return companies_and_tickers[companies_and_tickers['symbol'].isin(not_found_in_db)]

def return_matches(first_li, second_li):
    return set(first_li) & set(second_li)

def insert(df, entries):
    """
        Takes a set of values and inserts them into a dataframe sequentially where the index is
        sequential and continuous.
    """
    try:
        df.loc[max(df.index) + 1] = entries
    except ValueError:
        df.loc[0] = entries
        
def make_list_of_tickers(csi_dataframe, engine):
    tickers_from_nasdaq_site = new_nasdaq_tickers_and_names(engine)
    ticker_list1 = tickers_from_nasdaq_site['symbol'].values.tolist()
    ticker_list2 = csi_dataframe['symbol'].values.tolist()
    return list(set(ticker_list1 + ticker_list2))

In [ ]:
# Create a filings dataframe from the edgar database filings table
filings_data = pd.read_sql_query('select * from filings;', con=engine)
filings_data = filings_data.rename(columns={'company': 'company_name'})


In [4]:
# Create a csi_data datagrame from csidata.com
csi_stock_data = pd.read_csv('http://www.csidata.com/factsheets.php?type=stock&format=csv')
csi_data = pd.DataFrame(csi_stock_data, columns=['csinumber', 'symbol', 'company_name', 'exchange', 'isactive', 'startdate', 'enddate'])
csi_data = csi_data.dropna()

In [ ]:
ticker_list = make_list_of_tickers(csi_data, engine)
ticker_cik_dict, no_matches = tickers_to_cik(ticker_list[1:10])